# Introduction

Hello, and welcome to obvs! This is a package that seeks to re-implement Google PAIR's [Patchscopes](https://arxiv.org/abs/2401.06102) paper as a PyPI package, to enable researchers to use the methods discussed. This tutorial will cover the following:
- What a Patchscope is 
- How to create a `Patchscope` object 
- Why is this useful?

# What is a Patchscope?

A Patchscope is a framework that allows researchers to examine the hidden layers of LLMs. It takes advantage of the fact that LLMs can generate human-readable text, using this ability to shed light on what's happening within the model's internal structures. This framework provides a flexible and modular way to reveal information hidden in LLMs' internal states.

By "hidden representations," we mean the intermediate data produced by LLMs during their forward pass. Patchscopes enable us to inspect these hidden layers by patching or replacing parts of the model's internal computations, then asking the model to explain or describe them in plain text. 

In particular, we recommend reading section 3.1 of the paper, which outlines the notation used by the authors. In summary:
- $n$ = number of source tokens 
- $m$ = number of target tokens 
- $S$ = source prompt
- $T$ = target prompt
- $M$ = source model
- $M^*$ = target model
- $L$ = number of layers in source model
- $L^*$ = number of layers in target model
- $l$ = index of source layer
- $l^*$ = index of target layer
- $i$ = position in tokenized source prompt
- $i^*$ = position in tokenized target prompt 
- $h_i^l$ = hidden representation at layer $l$, position $i$ in source model
- $h_{i^*}^{l^*}$ = hidden representation at layer $l^*$, position $i^*$ in target model
- $f(h_i^l)$ = function applied to hidden representation from source model
- $f(h_{i^*}^{l^*})$ = function applied to hidden representation from target model
 
 With this in mind, we can define two important quantities:
 - A hidden representation from a source model is entirely defined by $(S, i, M, l)$
 - A Patchscope, acting upon that that hidden representation, is entirely defined by $(T, i^*, f, M^*, l^*)$

One can perhaps now envisage how a `Patchscope` object could be assembled in code:
- Within `obvs/patchscope_base.py`, we define an abstract `PatchscopeBase` class, with methods and attributes which should belong to any and all patchscopes (recovering source and token positions, output logits from the target model, the number of layers in both the source and target model, and defining what a forward pass looks like in the source and target model, as a few examples)
- Within `obvs/patchscope.py` , we create dataclasses for the source and target contexts, a `ModelLoader` class which uses `nnsight` to allow a handful of models in `obvs`, and finally a `Patchscope` class, which builds out some of the functionality outlined in the `PatchscopeBase` class. 
- We then build on this once more, to utilise the `Patchscope` class to finally recover some of the lenses that are said to be re-implementable as a Patchscope object. `lenses.py` contains code for a `PatchscopeLogitLens` object, as well as a `ClassicLogitLens` for comparison. It also contains a `TokenIdentity` lens (a method introduced in the paper), which uses clever prompt generation to recover the information stored in the hidden state of a model's forward pass. An implementation of a `FutureLens` is also demonstrated in `scripts/future_lens.py`. 


# How to create a Patchscope

Above we mentioned a hidden representation being entirely defined by $(S, i, M, l)$, and a Patchscope acting on that hidden representation being entirely defined by $(T, i^*, f, M^*, l^*)$. Given that the hidden representation originates from the source model $S$, let us instead refer to $(S, i, M, l)$ as the source context. Similarly, given that this is patched into the target model $T$, let us refer to $(T, i^*, f, M^*, l^*)$ as the target context. 

We're able to construct a `Patchscope` in `obvs` by first defining both a `SourceContext` and a `TargetContext` object (as defined in `patchscope.py`). Let's do this now:

In [ ]:
from obvs.patchscope import Patchscope, SourceContext, TargetContext

model_name = "gpt2"
prompt = "The cat sat on the hat "
position = 3
layer = 6
source_context = SourceContext(prompt = prompt,
                               model_name = model_name,
                               position = position,
                               layer = layer)

A few notes:
- Models are loaded via `nnsight`, using the `ModelLoader` class in `patchscope.py`
- The prompt doesn't need to be tokenized at this stage

Let's now create a `TargetContext` object:

In [ ]:
target_prompt = "The cat sat on the hat "
target_position = 5
target_layer = 8

target_context = TargetContext(prompt=target_prompt,
                               model_name=model_name,
                               position=target_position,
                               layer=target_layer,
                               )

Let's now combine these to create a `Patchscope` object:

In [ ]:
patchscope = Patchscope(source_context, target_context)

Let's now run the patchscope intervention and predict some subsequent tokens:

In [ ]:
patchscope.run()
patchscope.full_output_tokens()

So what did we just do? Well:
- We defined a prompt and intervened on its forward pass through GPT-2, at layer 6 and position 3.
- We extracted the hidden representation of the token at (position 3, layer 6) from the residual stream.
- We patched it into a second forward pass of GPT-2 (as defined by the `TargetContext`), at position 5 and layer 8.
- We continued this second forward pass to the end, and output some predicted tokens.

Now, why might this be useful? It doesn't seem immediately clear from this. 

# Why is this useful?

A Patchscope represents an abstraction of multiple mechanistic interpretability methods, across a number of tasks. The goal with this library is to make all of these methods available under one nice API. In terms of which methods are available, and for which tasks, this is summarised nicely in Table 1 of the [Patchscopes paper](https://arxiv.org/pdf/2401.06102). 
Note the notation used in the table. The key idea is that by making some small adjustments in the values of certain variables, one can recover different mechanistic interpretability techniques quite quickly.

# Next Steps

We've managed to use the `Patchscope` object to build a few lens classes in `obvs/lenses.py` . We've also created a few experiments in `scripts/`, including replicating some of the results from Figure 2 of the Patchscopes paper. We hope these serve as inspiration for how you may build on top of the `Patchscope` API. 

Best of luck, and we hope you have fun! 

The `obvs` team.